# Interpretable Clustering with Latent Dirichlet Allocation

In [1]:
import numpy as np
import pandas as pd
import pyLDAvis.gensim
from gensim import corpora
from gensim.models import LdaModel
from sklearn.metrics import normalized_mutual_info_score as NMI
pyLDAvis.enable_notebook()

/Users/taewook/anaconda/lib/python3.5/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In this tutorial,

I will show that the combination of LDA(Latent Dirichlet Allocation) model + pyLDAVis

can be powerful method for interpretable clustering.

## 1. Load Data

In [2]:
data = pd.read_csv('iris.csv')
dummy_data = data.copy()

In [3]:
class_var = 'species'
input_vars = [var for var in data.columns if var != class_var]

In [4]:
data[class_var].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

#### Perform clustering with #cluster = 3

In [5]:
n_cluster = 3

## 2. Discretize Continuous Variables

In [6]:
for var in input_vars:
    dummy_data[var] = pd.cut(data[var], bins=3)

In [7]:
dummy_data.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,"(4.296, 5.5]","(2.8, 3.6]","(0.994, 2.967]","(0.0976, 0.9]",setosa
1,"(4.296, 5.5]","(2.8, 3.6]","(0.994, 2.967]","(0.0976, 0.9]",setosa
2,"(4.296, 5.5]","(2.8, 3.6]","(0.994, 2.967]","(0.0976, 0.9]",setosa


## 3. Preprocess data to be input format of gensim LDA model

#### Convert Each Row to Token List

In [8]:
rows = []
for _, row in dummy_data.iterrows():
    rows.append(['{}_{}'.format(k,v) for k,v in row.items() if k != class_var])

In [9]:
rows[0]

['sepal_length_(4.296, 5.5]',
 'sepal_width_(2.8, 3.6]',
 'petal_length_(0.994, 2.967]',
 'petal_width_(0.0976, 0.9]']

#### Prepare dictionary for values of variables & Convert Structured Data into Corpus Format

In [10]:
dictionary = corpora.Dictionary(rows)
row_var_matrix = [dictionary.doc2bow(row) for row in rows]

In [11]:
dictionary.id2token

{}

In [12]:
rows[0]

['sepal_length_(4.296, 5.5]',
 'sepal_width_(2.8, 3.6]',
 'petal_length_(0.994, 2.967]',
 'petal_width_(0.0976, 0.9]']

In [13]:
row_var_matrix[0]

[(0, 1), (1, 1), (2, 1), (3, 1)]

## 4. Perform clustering with LDA model & Interpreting Clusters

In [14]:
lda = LdaModel(row_var_matrix, num_topics=n_cluster, id2word = dictionary, passes=50)

In [15]:
clustering_result = pyLDAvis.gensim.prepare(lda, row_var_matrix, dictionary)

#### With PyLdaVis, we can explore each cluster interactively!

In [16]:
clustering_result

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      36.098440        1       1  0.206889  0.193043
1      34.693117        1       2 -0.330100  0.035634
0      29.208442        1       3  0.123211 -0.228677, topic_info=     Category       Freq                         Term      Total  loglift  \
term                                                                        
10    Default  47.000000       petal_width_(1.7, 2.5]  47.000000  12.0000   
2     Default  47.000000    petal_length_(4.933, 6.9]  47.000000  11.0000   
1     Default  58.000000    sepal_length_(4.296, 5.5]  58.000000  10.0000   
4     Default  53.000000  petal_length_(2.967, 4.933]  53.000000   9.0000   
7     Default  53.000000       petal_width_(0.9, 1.7]  53.000000   8.0000   
6     Default  49.000000    petal_width_(0.0976, 0.9]  49.000000   7.0000   
0     Default  49.000000  petal_length_(0.994, 2.967]  49.000000   6.0000   
8     Default  46.000000     sepal_width_(1.998, 2.8]  46.000000   5.0000   
5     Default  70.000000      sepal_length_(5.5, 6.7]  70.000000   4.0000   
3     Default  21.000000      sepal_length_(6.7, 7.9]  21.000000   3.0000   
11    Default  15.000000       sepal_width_(3.6, 4.4]  15.000000   2.0000   
9     Default  87.000000       sepal_width_(2.8, 3.6]  87.000000   1.0000   
4      Topic1  52.339796  petal_length_(2.967, 4.933]  53.012846   1.0061   
7      Topic1  52.338844       petal_width_(0.9, 1.7]  53.012885   1.0061   
8      Topic1  43.292196     sepal_width_(1.998, 2.8]  46.380328   0.9500   
5      Topic1  45.836539      sepal_length_(5.5, 6.7]  70.507674   0.5883   
9      Topic1  19.577053       sepal_width_(2.8, 3.6]  87.615697  -0.4797   
1      Topic1   1.235005    sepal_length_(4.296, 5.5]  58.422780  -2.8377   
11     Topic1   0.324177       sepal_width_(3.6, 4.4]  15.590226  -2.8542   
3      Topic1   0.335452      sepal_length_(6.7, 7.9]  21.191472  -3.1270   
2      Topic1   0.333628    petal_length_(4.933, 6.9]  47.464124  -3.9388   
10     Topic1   0.332787       petal_width_(1.7, 2.5]  47.464168  -3.9413   
6      Topic1   0.322574    petal_width_(0.0976, 0.9]  49.668898  -4.0179   
0      Topic1   0.322574  petal_length_(0.994, 2.967]  49.668898  -4.0179   
6      Topic2  49.008740    petal_width_(0.0976, 0.9]  49.668898   1.0452   
0      Topic2  49.008740  petal_length_(0.994, 2.967]  49.668898   1.0452   
1      Topic2  56.849784    sepal_length_(4.296, 5.5]  58.422780   1.0313   
11     Topic2  14.880028       sepal_width_(3.6, 4.4]  15.590226   1.0120   
9      Topic2  36.106698       sepal_width_(2.8, 3.6]  87.615697   0.1721   
3      Topic2   0.326211      sepal_length_(6.7, 7.9]  21.191472  -3.1152   
8      Topic2   0.332316     sepal_width_(1.998, 2.8]  46.380328  -3.8799   
10     Topic2   0.325685       petal_width_(1.7, 2.5]  47.464168  -3.9232   
2      Topic2   0.325683    petal_length_(4.933, 6.9]  47.464124  -3.9232   
7      Topic2   0.328279       petal_width_(0.9, 1.7]  53.012885  -4.0258   
4      Topic2   0.328271  petal_length_(2.967, 4.933]  53.012846  -4.0258   
5      Topic2   0.338281      sepal_length_(5.5, 6.7]  70.507674  -4.2810   
10     Topic3  46.805696       petal_width_(1.7, 2.5]  47.464168   1.2167   
2      Topic3  46.804813    petal_length_(4.933, 6.9]  47.464124   1.2167   
3      Topic3  20.529809      sepal_length_(6.7, 7.9]  21.191472   1.1990   
9      Topic3  31.931946       sepal_width_(2.8, 3.6]  87.615697   0.2214   
5      Topic3  24.332854      sepal_length_(5.5, 6.7]  70.507674   0.1668   
8      Topic3   2.755816     sepal_width_(1.998, 2.8]  46.380328  -1.5924   
11     Topic3   0.386021       sepal_width_(3.6, 4.4]  15.590226  -2.4678   
0      Topic3   0.337585  petal_length_(0.994, 2.967]  49.668898  -3.7606   
6      Topic3   0.337584    petal_width_(0.0976, 0.9]  49.668898  -3.7606   
7      Topic3   0.345762       petal_width_(0.9, 1.7]  53.012885  -3

#### To enhance interpretability, merge adjacent intervals

In [17]:
def merge_adjacent_intervals(intervals_for_var):
    
    def get_begin_and_end_points(interval):
        return [float(x) for x in interval.split('_(')[1].replace(']','').split(', ')]
    
    var_name = intervals_for_var[0].split('_(')[0]
    intervals_for_var = sorted([get_begin_and_end_points(i) for i in intervals_for_var])
    
    prev_num_intervals = len(intervals_for_var)
    
    while(True):
        
        for idx, interval in enumerate(intervals_for_var):
            if idx < len(intervals_for_var) - 1:
                curr_interval = intervals_for_var[idx]
                next_interval = intervals_for_var[idx+1]
                if curr_interval[1] == next_interval[0]:
                    merged_interval = [curr_interval[0], next_interval[1]]
                    intervals_for_var[idx+1] = merged_interval
                    intervals_for_var.remove(curr_interval)
                
        curr_num_intervals = len(intervals_for_var)
        if prev_num_intervals > curr_num_intervals:
            prev_num_intervals = curr_num_intervals
        else:
            break
    
    intervals_for_var = ['{}_({}, {}]'.format(var_name, i[0], i[1]) for i in intervals_for_var]
    return intervals_for_var

def postprocess_intervals(var_names, intervals):
    intervals_by_var = []
    for var in var_names:
        intervals_for_var = [x for x in intervals if var == x[:len(var)]]
        if len(intervals_for_var) >= 2:
            intervals_for_var = merge_adjacent_intervals(intervals_for_var)
        intervals_by_var.append(intervals_for_var)
    return intervals_by_var

#### Now, we can describe clusters in terms of intervals of variables

In [18]:
cutoff = 0.1 # P(value of variable | cluster) (Same as P(word | topic) in LDA Model)
for i in range(n_cluster):
    variables = postprocess_intervals(input_vars,
                                       [dictionary.id2token[x[0]] for x in lda.get_topic_terms(i) if x[1] > cutoff])
    print('{} th cluster'.format(i), variables)

0 th cluster [['sepal_length_(5.5, 7.9]'], ['sepal_width_(2.8, 3.6]'], ['petal_length_(4.933, 6.9]'], ['petal_width_(1.7, 2.5]']]
1 th cluster [['sepal_length_(4.296, 5.5]'], ['sepal_width_(2.8, 3.6]'], ['petal_length_(0.994, 2.967]'], ['petal_width_(0.0976, 0.9]']]
2 th cluster [['sepal_length_(5.5, 6.7]'], ['sepal_width_(1.998, 2.8]'], ['petal_length_(2.967, 4.933]'], ['petal_width_(0.9, 1.7]']]


## 5. Clustering performance evaluation

#### Get cluster(=topic) label of each row

In [19]:
def get_cluster_label(lda, row):
    return np.argmax([x[1] for x in lda.get_document_topics(row)])

In [20]:
cluster_labels = [get_cluster_label(lda, row) for row in row_var_matrix]
NMI(data.species, cluster_labels)

0.8243868199386184

#### Compare NMI with K-means clustering

In [21]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_cluster)
cluster_labels_kmeans = kmeans.fit_predict(data.drop(class_var, axis=1))
print(NMI(data.species, cluster_labels_kmeans))

0.758205727819


#### Try K-means with normalized input

In [22]:
scaled_data = data.drop(class_var, axis=1)
scaled_data = (scaled_data - scaled_data.mean(axis=0)) / scaled_data.std()
cluster_labels_kmeans_scaled = kmeans.fit_predict(scaled_data)
print(NMI(data.species, cluster_labels_kmeans_scaled))

0.659486991304


Clustering performance of proposed method outperforms K-means clustering,

and is able to provide explanation for each cluster at the same time.